In [1]:
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas as pd
import seaborn as sns
from tslearn.clustering import TimeSeriesKMeans
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN 
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
import seaborn as sns
import pickle
import functools
import fastdtw
import scipy.spatial.distance as sd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

2022-06-09 18:13:04.331827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 18:13:04.331902: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
consumption_df = pd.read_csv('Final Datasets/energy_consumption.csv')
no_scale = consumption_df[['Unnamed: 0', 'timestamp',  'House Number']]
consumption_df = consumption_df[['energy_kWh','electronics workbench energy', 'network equipment energy',
       'home office energy', 'RAE home office energy',
       'RAE Mobile Phone Charger Energy']]

no_scale

,Unnamed: 0,timestamp,House Number
0,0,2012-06-01 01:00:00,1
1,1,2012-06-01 02:00:00,1
2,2,2012-06-01 03:00:00,1
3,3,2012-06-01 04:00:00,1
4,4,2012-06-01 05:00:00,1
...,...,...,...
611013,123761,2020-05-19 19:00:00,28
611014,123762,2020-05-19 20:00:00,28
611015,123763,2020-05-19 21:00:00,28
611016,123764,2020-05-19 22:00:00,28


In [3]:
consumption_df

,energy_kWh,electronics workbench energy,network equipment energy,home office energy,RAE home office energy,RAE Mobile Phone Charger Energy
0,1.0110,0.008686,0.019316,0.018232,0.134570,0.000431
1,0.4510,0.007178,0.019338,0.013745,0.061839,0.000215
2,0.5050,0.007324,0.019336,0.014177,0.068852,0.000236
3,0.4410,0.007151,0.019339,0.013664,0.060540,0.000211
4,0.4680,0.007224,0.019338,0.013881,0.064047,0.000221
...,...,...,...,...,...,...
611013,3.5802,0.028408,0.038470,0.069307,0.801381,0.002441
611014,3.4398,0.027762,0.038479,0.067384,0.770210,0.002348
611015,2.3049,0.022537,0.038556,0.051836,0.518247,0.001601
611016,2.4921,0.023399,0.038544,0.054401,0.559808,0.001724


In [4]:
generation_df = pd.read_csv('Final Datasets/energy_generation.csv')
no_scale_generation_df = generation_df[['timestamp']]
generation_df = generation_df[['Total Generated Energy']]

In [5]:
no_scale_generation_df

,timestamp
0,2012-01-01 00:00:00
1,2012-01-01 01:00:00
2,2012-01-01 02:00:00
3,2012-01-01 03:00:00
4,2012-01-01 04:00:00
...,...
78907,2020-12-31 19:00:00
78908,2020-12-31 20:00:00
78909,2020-12-31 21:00:00
78910,2020-12-31 22:00:00


In [6]:
generation_df

,Total Generated Energy
0,0.168265
1,0.168265
2,0.168265
3,0.168265
4,0.168265
...,...
78907,1.710441
78908,1.710441
78909,1.497873
78910,1.285454


In [10]:
scaler = StandardScaler()
scaled = scaler.fit_transform(consumption_df)
standard_scalar_df = pd.DataFrame(scaled, columns=['energy_kWh','electronics workbench energy', 'network equipment energy',
       'home office energy', 'RAE home office energy',
       'RAE Mobile Phone Charger Energy'])
standard_scalar_df[['Unnamed: 0', 'timestamp',  'House Number']] = no_scale
standard_scalar_df
standard_scalar_df.to_csv('Final Datasets/energy_consumption_standardized.csv')

In [12]:
generation_df

,Total Generated Energy
0,0.168265
1,0.168265
2,0.168265
3,0.168265
4,0.168265
...,...
78907,1.710441
78908,1.710441
78909,1.497873
78910,1.285454


In [22]:
names = ['b', 'c', 'd', 'e', 'f'] #names for the columns
for i in names:
    generation_df[i] = generation_df['Total Generated Energy'] #assigns the columns in a loop
generation_scaled = scaler.transform(generation_df)
scalar_generation_df = pd.DataFrame(generation_scaled, columns=['Total Generated Energy','b', 'c', 'd', 'e', 'f'])
scalar_generation_df['timestamp'] = no_scale_generation_df
scalar_generation_df = scalar_generation_df[['timestamp', 'Total Generated Energy']]
scalar_generation_df

,timestamp,Total Generated Energy
0,2012-01-01 00:00:00,-0.779400
1,2012-01-01 01:00:00,-0.779400
2,2012-01-01 02:00:00,-0.779400
3,2012-01-01 03:00:00,-0.779400
4,2012-01-01 04:00:00,-0.779400
...,...,...
78907,2020-12-31 19:00:00,0.967615
78908,2020-12-31 20:00:00,0.967615
78909,2020-12-31 21:00:00,0.726812
78910,2020-12-31 22:00:00,0.486180


In [23]:
scalar_generation_df.to_csv('Final Datasets/energy_generation_standardized.csv')

In [19]:
# scalar_generation_df 